# Помогите Васе обмануть продавщицу

Вася очень любит апельсины, бананы, яблоки, мандарины и грейпфруты. Слава богу, в городе, где живёт Вася, целых 10400 магазинов фруктов, и везде текущие цены на фрукты разные и независимые друг от друга.

В каждом магазине города продается талончик, позволяющий в любой выходной в ближайшие 10 лет прийти в этот магазин, предъявить талончик, заплатить 100 рублей и получить 1 килограмм любого фрукта в этом магазине.

Вася написал сложную функцию $f(a_1,a_2,a_3,a_4,a_5)$, которая вычисляет справедливую стоимость этого талончика в конкретном магазине. Он считает, что все цены ведут себя как независимые броуновские движения с одинаковыми волатильностями, равными 10, и он знает только их начальные значения в конкретном магазине.
То есть $$x_i(t) = W_i(t) + x_i(0).$$
Здесь $x_i(0) = a_i$ — цена $i$-ого фрукта в данном магазине в начальный момент времени, $x_i(t)$ — цена $i$-ого фрукта в данном магазине в момент времени $t$. О том, каков экономический смысл функции $f$, написано в примечаниях.

Из всех магазинов Вася хочет сегодня купить самый выгодный талончик, но для этого ему нужно посчитать $f(a_1,a_2,a_3,a_4,a_5)$ для всех магазинов, а вычисления работают очень медленно.

Вам предстоит построить аппроксимацию функции, работающую достаточно быстро, чтобы заполнить ей тестовые значения функции. Изначально вам дано 50 обучающих точек и результаты функции, которые Вася успел посчитать, далее вы можете вызывать функцию в любых точках (в том числе и тестовых) самостоятельно.


**Замечание 1.** Обратите внимание, что функция выдает близкие, но не одинаковые результаты для одинаковых аргументов. В ответах, с которыми будет сравниваться ваша аппроксимация, функция посчитана 8 раз в каждой точке и усреднена.

**Замечание 2.** Формально, справедливая стоимость талончика — это цена бермудского опциона на максимум из $x_i$ со страйком $100$ и с постоянной ставкой дисконтирования $0.1$. Держатель опциона выбирает оптимальный момент, когда ему купить самый дорогой фрукт за $100$ рублей (или не покупать вообще): $$f(a_1, a_2, a_3, a_4, a_5) = \sup\limits_{\tau\in\mathcal{T}}\mathbb{E}\left[e^{-0.1\tau}\max(0, \max\limits_i(x_i({\tau})) - 100)\hspace{0.1cm}|\hspace{0.1cm} \forall i: x_i(0)=a_i\right].$$

**Замечание 3.** Ваше решение будет оцениваться по метрике RMSE, которая вычисляется по формуле
$$
\sqrt{\frac{\sum(y_{true}-y_{pred})^2}{n}}
$$

Ваш итоговый балл равен:
$$
\max\left(0,\frac{(max\_rmse-rmse\_score)\cdot 20}{max\_rmse}\right)
$$

где $max\_rmse=100$.
После окончания контеста баллы прошкалируются в соответствии с баллом наилучшего решения.

## Функция

Ниже определена функция f, как-либо изменять реализацию не требуется.
Пример вызова функции:
```
import numpy as np
result = f(
    np.array(
        [100.0, 100.0, 100.0, 100.0, 100.0]
    )
)
```

In [ ]:
!pip install numba --quiet
!pip install scipy --quiet

In [ ]:
from numba import njit
import numpy as np
from warnings import filterwarnings

filterwarnings("ignore")


@njit
def sample_wiener_process_one_path(
        time_deltas: np.ndarray,
        sigma: float
) -> np.ndarray:
    """
      Generates one realization of a Wiener process (Brownian motion).

      Parameters:
      - time_deltas (np.ndarray): Array of time intervals between grid points = np.diff(time_grid).
      - sigma (float): The volatility (standard deviation) of the Wiener process.

      Returns:
      np.ndarray: An array representing the generated Wiener process path.

      Note: This function utilizes the `numba` library's JIT (Just-In-Time) compilation for improved
      performance. Ensure `numba` is installed in your environment.
    """
    normals = np.random.normal(0, sigma, len(time_deltas))
    result = np.zeros(len(time_deltas) + 1, dtype=float)
    result[1:] = np.cumsum(normals * np.sqrt(time_deltas))
    return result


@njit
def sample_wiener_process(
        number_of_simulations: int,
        time_deltas: np.ndarray,
        sigma: float
) -> np.ndarray:
    """
      Generates some realizations of a Wiener process (Brownian motion).

      Parameters:
      - number_of_simulations (int): number of paths to generate
      - time_deltas (np.ndarray): Array of time intervals between successive observations.
      - sigma (float): The volatility (standard deviation) of the Wiener process.

      Returns:
      np.ndarray: An array, each row representing the generated Wiener process trajectory.
    """
    result = np.zeros((number_of_simulations, len(time_deltas) + 1), dtype=float)
    for i in range(number_of_simulations):
        result[i] = sample_wiener_process_one_path(time_deltas, sigma)
    return result


def transform_to_monomials_degree_3(
        x: np.ndarray
) -> np.ndarray:
    """
      Transforms input features to all monomials degree <= 3

      Parameters:
      - x (np.ndarray): Input feature matrix of shape (n_samples, n_features).

      Returns:
      - transformed (np.ndarray): Array to store the transformed features, shape (n_samples, (n_features + 1) * (n_features + 2) // 2).
    """
    index = 0
    transformed = np.zeros((x.shape[0],
                            (x.shape[1] + 1) * (x.shape[1] + 2) // 2 + (x.shape[1] + 1) * (x.shape[1] + 2) * (
                                    x.shape[1] + 3) // 6
                            ), dtype=float)
    # triple-wise product
    for i in range(x.shape[1]):
        for j in range(i, x.shape[1]):
            for k in range(j, x.shape[1]):
                transformed.T[index] = x.T[i] * x.T[j] * x.T[k]
                index += 1
    # pairwise product
    for i in range(x.shape[1]):
        for j in range(i, x.shape[1]):
            transformed.T[index] = x.T[i] * x.T[j]
            index += 1
    # repeat features
    for i in range(x.shape[1]):
        transformed.T[index] = x.T[i]
    # constant feature
    transformed.T[-1] = 1.
    return transformed


def f(assets0: np.ndarray) -> float:
    """
      Calculates bermudan call option with underlying max(asset_1, asset_2, ...) and strike_t = 0, asset_i --- Wiener process

      Parameters:
      - assets0 (np.ndarray): assets at the time 0
      Returns:
      float: option price

      time_grid = np.linspace(0., 1., 31)
      sigmas = 10.
      number_of_simulations = 2 ** 13
      strike = 100.
    """
    filterwarnings("ignore")
    # init
    try:
        number_of_simulations = 2 ** 13
        t = 10
        time_grid = np.array(list(range(365 * t + 1)))
        time_grid = time_grid[np.where(np.logical_or(time_grid % 7 == 1, time_grid % 7 == 0))] / 365
        time_deltas = np.diff(time_grid)
        number_of_steps = len(time_deltas)
        discount_rate = 0.1

        # sampling
        discount_factor = np.exp(-discount_rate * time_grid)
        strike = np.ones(number_of_steps + 1, dtype=float) * 100. * discount_factor
        number_of_assets = len(assets0)
        sigmas = np.ones_like(assets0, dtype=float) * 10.
        assets = np.zeros((number_of_assets, number_of_simulations, number_of_steps + 1), dtype=float)
        for i in range(len(sigmas)):
            assets[i] = (sample_wiener_process(number_of_simulations, time_deltas, sigmas[i]) + assets0[
                i]) * discount_factor

        # AMC
        option_price = np.clip(np.max(assets[:, :, -1], axis=0) - strike[-1], 0, 1e15)
        regularization_alpha = 1e-2
        weights = [None] * (number_of_steps + 1)
        for time_index in range(number_of_steps, -1, -1):
            current_payoff = np.clip(np.max(assets[:, :, time_index], axis=0) - strike[time_index], 0, 1e15)
            if time_index == 0:
                continuation_value = np.ones(number_of_simulations) * np.mean(option_price)
                in_the_money_indices = np.arange(number_of_simulations, dtype=int)
            else:
                in_the_money_indices = np.where(current_payoff > 1e-6)[0]
                if len(in_the_money_indices) < 20:
                    continue
                features = assets[:, in_the_money_indices, time_index].T
                transformed = transform_to_monomials_degree_3(features)
                regularization = np.eye(transformed.shape[1], dtype=float) * regularization_alpha
                inv = np.linalg.pinv((transformed.T @ transformed + regularization), rcond=1e-4)
                weights[time_index] = inv @ transformed.T @     option_price[in_the_money_indices].reshape(-1, 1)
                continuation_value = transformed @ weights[-1]

            indicator = current_payoff[in_the_money_indices] > continuation_value.reshape(-1)
            option_price[in_the_money_indices] = \
                indicator * current_payoff[in_the_money_indices] + (~indicator) * continuation_value.reshape(-1)

        for i in range(len(sigmas)):
            assets[i] = (sample_wiener_process(number_of_simulations, time_deltas, sigmas[i]) + assets0[
                i]) * discount_factor

        option_price = np.clip(np.max(assets[:, :, -1], axis=0) - strike[-1], 0, 1e15)
        for time_index in range(number_of_steps, -1, -1):
            current_payoff = np.clip(np.max(assets[:, :, time_index], axis=0) - strike[time_index], 0, 1e15)
            if time_index == 0:
                continuation_value = np.ones(number_of_simulations) * np.mean(option_price)
                in_the_money_indices = np.arange(number_of_simulations, dtype=int)
            else:
                in_the_money_indices = np.where(current_payoff > 1e-6)[0]
                if len(in_the_money_indices) < 20 or weights[time_index] is None:
                    continue
                features = assets[:, in_the_money_indices, time_index].T
                transformed = transform_to_monomials_degree_3(features)
                continuation_value = transformed @ weights[time_index]

            indicator = current_payoff[in_the_money_indices] > continuation_value.reshape(-1)
            option_price[in_the_money_indices] = \
                indicator * current_payoff[in_the_money_indices] + (~indicator) * continuation_value.reshape(-1)

        return np.mean(option_price)
    except np.linalg.LinAlgError:
        return f(assets0)

## Бейзлайн

In [ ]:
!pip install catboost --quiet

## Загружаем обучающие данные

In [ ]:
import pandas as pd

In [ ]:
train_points = pd.read_csv("train_points.csv", index_col=False)
print(f"Всего {len(train_points)} точек в train")
train_points.head()

Всего 50 точек в train


,a_1,a_2,a_3,a_4,a_5
0,62.690480,99.988546,82.652142,136.951889,108.831765
1,83.138739,90.577900,148.840644,122.974943,92.700708
2,120.593993,96.246944,56.809250,56.060180,105.655439
3,86.641013,101.944141,120.224920,110.265302,93.997373
4,72.594457,61.785805,99.923973,89.100706,127.304230


In [ ]:
train_answers = pd.read_csv("train_answers.csv", index_col=False)
train_answers = train_answers["f"]
train_answers.head()

0    247.992826
1    319.679662
2    180.894055
3    188.396445
4    195.280624
Name: f, dtype: float64

## Загружаем тестовые данные

In [ ]:
test_points = pd.read_csv("test_points.csv", index_col=False)
print(f"Всего {len(test_points)} точек в test")
test_points.head()

Всего 10000 точек в test


,a_1,a_2,a_3,a_4,a_5
0,102.146600,98.596161,79.045105,94.008758,74.269817
1,111.148611,109.187352,99.630397,92.308625,103.215788
2,115.449171,94.012153,107.968172,98.404978,95.377559
3,76.481240,99.627790,115.317668,78.711992,113.100374
4,88.562192,83.049915,95.263844,85.751793,125.281651


## Выделим val из обучающих данных
Выборка совсем маленькая, но хочется получить хоть какую-то оценку нашего качества.

In [ ]:
from sklearn.model_selection import train_test_split
train_points, val_points, train_answers, val_answers = train_test_split(points, train_answers, train_size=0.5)

## Бейзлайн: обучаемся только на train

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error

model = RandomForestRegressor(n_estimators=100)

model.fit(train_points, train_answers)
rmse = root_mean_squared_error(model.predict(val_points), val_answers)
std = val_answers.std()

print(f"RMSE: {rmse.round(5)}, Standard deviation: {std.round(5)}")

RMSE: 41.63415, Standard deviation: 57.78981


### Скор, который мы получим такой посылкой в контесте
**Вы этот код выполнить не сможете, так как доступ к файлу `test_answers.csv` есть только у авторов.**

In [ ]:
test_answers = pd.read_csv("../test_answers.csv", index_col=False)
test_answers = test_answers["f"]

rmse = root_mean_squared_error(model.predict(test_points), test_answers)
std = test_answers.std()

print(f"RMSE: {rmse.round(5)}, Standard deviation: {std.round(5)}")

RMSE: 47.42411, Standard deviation: 61.22109


Видим, что получилось ОЧЕНЬ плохо. Кроме того, видимо, объём `val` СЛИШКОМ маленький для измерения качества модели. Вы можете менять его, как вам угодно.

## Вычислим значения функции $f$ на дополнительных 50 точках из test.

In [ ]:
!pip install tqdm --quiet
from tqdm import tqdm

In [ ]:
additional_train_points = test_points[:50]
additional_train_answers = [
    f(item) for item in tqdm(additional_train_points.values)
]

100%|███████████████████████████████████████████| 50/50 [12:06<00:00, 14.54s/it]


## Проведём обучение для обогащённой выборки

In [ ]:
enriched_train_points = pd.concat([train_points, additional_train_points])
enriched_train_answers = pd.concat([train_answers, pd.Series(additional_train_answers, name="f")])

In [ ]:
model = RandomForestRegressor(n_estimators=100)

model.fit(enriched_train_points, enriched_train_answers)
rmse = root_mean_squared_error(model.predict(val_points), val_answers)
std = val_answers.std()

print(f"RMSE: {rmse.round(5)}, Standard deviation: {std.round(5)}")

RMSE: 27.25497, Standard deviation: 57.78981


### Скор, который мы получим такой посылкой в контесте
**Вы этот код выполнить не сможете, так как доступ к файлу `test_answers.csv` есть только у авторов.**

In [ ]:
rmse = root_mean_squared_error(model.predict(test_points), test_answers)
std = test_answers.std()

print(f"RMSE: {rmse.round(5)}, Standard deviation: {std.round(5)}")

RMSE: 37.00382, Standard deviation: 61.22109


Уже получилось гораздо лучше. Но в контесте эта посылка всё равно даст 0 баллов и вердикт WA, поскольку баллы выдаются с $rmse=25$.

## Формируем sample_submission

In [ ]:
def create_submission(model, test_points, out_file="sample_submission.csv"):
    test_answers = model.predict(test_points)
    test_answers = pd.Series(test_answers, name="f").to_csv(out_file, index=None)

In [ ]:
create_submission(model, test_points)